# Processing Zipped Files

Data engineering is a critical component to any successful data analysis project. In this project, we explore the use of a for loop in Python to unzip each file one at a time and use the CSV sniffer to determine the delimiter and whether or not there is a header row. By doing this, we are able to effectively process the data and create a comprehensive data analysis.

Additionally this project helped my repo here <link> 

In [1]:
import datetime
import csv
import pandas as pd 
import numpy as np

import os
import re
import io

import zipfile
import janitor
from zipfile import ZipFile

## Create an empty data frame called wedge_summary with the following columns: file_name, num_rows, num_cards, num_dates

In [11]:
wedge_summary = pd.DataFrame({'file_name' : pd.Series(dtype='str'),
                            'num_rows' : pd.Series(dtype = 'int'),
                             'num_cards' : pd.Series(dtype = 'int'),
                             'num_dates' : pd.Series(dtype = 'int')})

In [12]:
wedge_summary

,file_name,num_rows,num_cards,num_dates


## Iterate over the zip files that hold the Wedge transaction files

In [13]:
#os for directory of files
zip_files = os.listdir("WZOZ/")

In [ ]:
#iterating over zipped files
for zipf in zip_files :
    with ZipFile("WZOZ/" + zipf,'r') as zf :
        print(zf.namelist())

In [15]:
#setting delimiter dictionary here 
delimiters = dict() 

# Start by reading in all the files again
for this_zf in zip_files :
    with ZipFile("WZOZ/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["It looks like",
                           file_name,
                           "has delimiter",
                           dialect.delimiter,
                           "."]))

            input_file.close() 

It looks like transArchive_201410_201412.csv has delimiter , .
It looks like transArchive_201301_201303_inactive.csv has delimiter ; .
It looks like transArchive_201210_201212.csv has delimiter , .
It looks like transArchive_201609.csv has delimiter , .
It looks like transArchive_201608.csv has delimiter , .
It looks like transArchive_201201_201203.csv has delimiter , .
It looks like transArchive_201204_201206.csv has delimiter , .
It looks like transArchive_201407_201409.csv has delimiter , .
It looks like transArchive_201207_201209.csv has delimiter , .
It looks like transArchive_201404_201406.csv has delimiter , .
It looks like transArchive_201401_201403.csv has delimiter , .
It looks like transArchive_201404_201406_inactive.csv has delimiter ; .
It looks like transArchive_201210_201212_inactive.csv has delimiter ; .
It looks like transArchive_201307_201309_inactive.csv has delimiter ; .
It looks like transArchive_201501_201503.csv has delimiter , .
It looks like transArchive_201307

In [16]:
header = dict()

#checking for headers here. 
for this_zf in zip_files :
    with ZipFile("WZOZ/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            this_delimiter = delimiters[file_name]
            
            for line in input_file :
                print(line.strip().split(this_delimiter))
                break

            header[file_name] = "datetime" in line

            input_file.close() # tidy up

['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicable"', '"discount"', '"memDiscount"', '"discountable"', '"discounttype"', '"voided"', '"percentDiscount"', '"ItemQtty"', '"volDiscType"', '"volume"', '"VolSpecial"', '"mixMatch"', '"matched"', '"memType"', '"staff"', '"numflag"', '"itemstatus"', '"tenderstatus"', '"charflag"', '"varflag"', '"batchHeaderID"', '"local"', '"organic"', '"display"', '"receipt"', '"card_no"', '"store"', '"branch"', '"match_id"', '"trans_id"']
['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicabl

In [17]:
headers = ['"datetime"', '"register_no"', '"emp_no"', '"trans_no"', '"upc"', '"description"', '"trans_type"', '"trans_subtype"', '"trans_status"', '"department"', '"quantity"', '"Scale"', '"cost"', '"unitPrice"', '"total"', '"regPrice"', '"altPrice"', '"tax"', '"taxexempt"', '"foodstamp"', '"wicable"', '"discount"', '"memDiscount"', '"discountable"', '"discounttype"', '"voided"', '"percentDiscount"', '"ItemQtty"', '"volDiscType"', '"volume"', '"VolSpecial"', '"mixMatch"', '"matched"', '"memType"', '"staff"', '"numflag"', '"itemstatus"', '"tenderstatus"', '"charflag"', '"varflag"', '"batchHeaderID"', '"local"', '"organic"', '"display"', '"receipt"', '"card_no"', '"store"', '"branch"', '"match_id"', '"trans_id"']

In [18]:
stripped = [i.replace('"', '') for i in headers]

In [ ]:
header = dict()

#setting empty list
files = []
rows = []
cards = []
date = []

#checking for headers here. 
for this_zf in zip_files :
    with ZipFile("WZOZ/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            this_delimiter = delimiters[file_name]
            
            for line in input_file :
                if header.values() == "True":
                    df = pd.read_csv(input_file, sep = delimiters[file_name], encoding="utf-8")
    
                else:
                    df = pd.read_csv(input_file, sep = delimiters[file_name], names = stripped, encoding="utf-8")

                files.append(file_name)
    
                num_cards = df.card_no.nunique()
                cards.append(num_cards)

                row_count = (df.shape[0])
                rows.append(row_count)

                df['date'] = pd.to_datetime(df['datetime']).dt.floor('D')
                num_date = df.date.nunique()
                date.append(num_date)
    
                break

            input_file.close() # tidy up

Now we have two dictionaries. One with the delimiters for the correlated files. And a dictionary for IF there is a header or not. 

Now we can read in the files so they are correctly delimited. 

## For each file, store a row in wedge_summary that holds the values listed above. num_cards should be the unique card numbers in the file and num_dates should be the number of dates.


In [21]:
wedge_summary.file_name = files

wedge_summary.num_cards = cards

wedge_summary.num_dates = date

wedge_summary.num_rows = rows

In [22]:
wedge_summary

,file_name,num_rows,num_cards,num_dates
0,transArchive_201410_201412.csv,2931416,12362,91
1,transArchive_201301_201303_inactive.csv,148623,1564,90
2,transArchive_201210_201212.csv,2893637,10710,91
3,transArchive_201609.csv,861247,9159,30
4,transArchive_201608.csv,858167,9130,31
5,transArchive_201201_201203.csv,2989644,10154,91
6,transArchive_201204_201206.csv,3083546,10413,91
7,transArchive_201407_201409.csv,3030409,12462,92
8,transArchive_201207_201209.csv,2925608,10551,92
9,transArchive_201404_201406.csv,3154267,12332,91
